<a href="https://colab.research.google.com/github/Haridasaravind/CS612/blob/main/CS612.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.1 MB/s eta 0:00:00


In [8]:
import os
import random
from Bio import SeqIO

def read_fasta(directory='.'):
    # Obtain a list of all files with the .fas extension in the specified directory
    fas_files = [f for f in os.listdir(directory) if f.endswith('.fas')]
    # Select a random .fas file from the list
    selected_file = random.choice(fas_files)
    print(f"Selected File: {selected_file}")

    # Read the multiple sequence alignment (MSA) file
    records = SeqIO.parse(os.path.join(directory, selected_file), "fasta")

    # Dictionary to store sequences grouped by family
    sequences_by_family = {}

    # Process each sequence record
    for record in records:
        # Check if the record ID contains an underscore
        if '_' in record.id:
            # Extract family information from the record ID
            family = record.id.split("_")[1]

            # Add the sequence to the corresponding family in the dictionary
            if family in sequences_by_family:
                sequences_by_family[family].append(record.seq)
            else:
                sequences_by_family[family] = [record.seq]
        else:
            print(f"Warning: Skipping record with ID '{record.id}' because it does not contain an underscore")

    # Print summary statistics
    total_sequences = sum(len(sequences) for sequences in sequences_by_family.values())
    print(f"Total sequences: {total_sequences}")
    
    # Uncomment the following code to print the unique families
    # print(f"Total unique families: {len(sequences_by_family)}")
    # for family, sequences in sequences_by_family.items():
    #     print(f"Family {family}: {len(sequences)} sequences")

    # Uncomment the following code to print the sequences grouped by family
    # for family, sequences in sequences_by_family.items():
    #     print(f"\nFamily: {family}")
    #     for sequence in sequences:
    #         print(sequence)
    
    return sequences_by_family

sequences = read_fasta()

print(sequences)

Selected File: YEFM_YOEB_20_id90.fas
Total sequences: 267
{'E0ELC3': [Seq('MQAITYTEARQNLAGTMSKVAEDFEPILITRSKGGNCVLMSYEQYSSLEETAYL...YHY')], 'F0TL51': [Seq('MFVASVSDFRKDIKSYFDRVTKNFETLIINRGKDSGIVVMSLDEYNSLMATNHE...FHY')], 'F9HHN3': [Seq('MEAVVYSNFRNNLKEYMKKVNDEYEPLMVVNKNPEDIVVLSKDNWDSIQETIRL...DHY')], 'C1DS88': [Seq('----------------MERVNDDRAPILITRQKGEPVVMMSLADYNAFEETAYL...YRY')], 'B5W4K1': [Seq('--ETTYSQARMNLATILDEVCDQRQIVVIKRPNEKNVALIAEDELESLLECVYL...YHY')], 'E6QCC1': [Seq('MHVVSYSEARDNLKAVMDRAVEDADVTIITRRGSSNAVLLSQDLFDSLMETVHL...YHY')], 'E6QQB2': [Seq('MDAMTYTTVRANLASTMDRVCNDHEALIITRNGEQAVVMLSLEDFKALEETAYL...FHY')], 'E4ILX7': [Seq('MEAVAYSNFRQNLRSYMKQVNEDAETLIVTSKDVETVVVLSKRDYDSMQETLRT...DHY')], 'D9WTC5': [Seq('-MPITASEARQNLFPLIEQVNEDHAPVHITSRKGNA-VLMSEEDFTSWTETVHL...YHY')], 'C6ZFQ6': [Seq('MQTVNYSTFRSELSDSMDRVTKNHSPMIVTRSKKEAVVMMSLEDFKAYEETAYL...YHY')], 'B9JZQ1': [Seq('MDTVFFSKARAELAGLLDKVNEDASAVEIVRRDKPSAVLMSKEEYESMVETLHL...YHY')], 'G2PLF3': [Seq('MQITSVSDFRKDIKTYLDRVVKDFETLVINRGKD

In [9]:
import random

def read_sequence(sequences):
    # Flatten the list of sequences
    all_sequences = [seq for seq_list in sequences.values() for seq in seq_list]
    # Select a random sequence from the list
    return random.choice(all_sequences)


In [12]:
seq = read_sequence(sequences)
print(seq)

MDVMTYSDARAQLKGVMDRAIHDKQEVIVTRKKGESVVVVSLETWNAVNETLHLLSTPKNASRLRASIAQLDAGTGEERELTEMKLVFSDHAWEDYQYWVSTNDKVRARINELIKQCKRTPFKGTGKPEPLKGDLTGWWSRRISQEDRMVYRVDSQSLEIAQLRFHY


In [14]:
import numpy as np

def one_hot_encoding(seq):
    # Define the amino acid alphabet
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    # Create a dictionary to map each amino acid to its index in the alphabet
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    # Initialize the one-hot encoded array
    one_hot = np.zeros((len(seq), len(amino_acids)))
    # Set the appropriate elements to 1
    for i, aa in enumerate(seq):
        if aa in aa_to_index:
            one_hot[i, aa_to_index[aa]] = 1
    return one_hot


In [15]:
one_hot = one_hot_encoding(seq)
print(one_hot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [16]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np

# Define the amino acid alphabet
amino_acids = 'ACDEFGHIKLMNPQRSTVWY-'

def one_hot_encoding(seq):
    # Create a dictionary to map each amino acid to its index in the alphabet
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    # Initialize the one-hot encoded array
    one_hot = np.zeros((len(seq), len(amino_acids)))
    # Set the appropriate elements to 1
    for i, aa in enumerate(seq):
        if aa in aa_to_index:
            one_hot[i, aa_to_index[aa]] = 1
    return one_hot

# Define the size of the latent space
latent_dim = 32

# Define the architecture of the autoencoder
input_seq = Input(shape=(None, len(amino_acids)))
encoded = Dense(latent_dim, activation='relu')(input_seq)
decoded = Dense(len(amino_acids), activation='softmax')(encoded)
autoencoder = Model(input_seq, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')


In [18]:
# Encode the sequence using one-hot encoding
seq_encoded = one_hot_encoding(seq).reshape(1, -1, len(amino_acids))

In [19]:
# Train the autoencoder on the encoded sequence
autoencoder.fit(seq_encoded, seq_encoded, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 892ms/step - loss: 3.0518
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 3.0450
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 3.0383
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 3.0316
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 3.0249
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 3.0182
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 3.0115
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 3.0048
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 2.9981
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 2.9914
Epoch 11/100
1/1 [==============================] - 0s 9ms/step - loss: 2.9847
Epoch 12/100
1/1 [==============================] - 0s 9ms/step - loss: 2.9780
Epoch 13/100
1/1 [==============================] - 0s

In [20]:
# Use the autoencoder to encode and decode the sequence
seq_decoded = autoencoder.predict(seq_encoded)

1/1 [==============================] - 0s 124ms/step


In [21]:
# Compute reconstruction error metrics
cross_entropy = -np.mean(np.sum(seq_encoded * np.log(seq_decoded), axis=-1))
same_aa_accuracy = np.mean(np.argmax(seq_encoded, axis=-1) == np.argmax(seq_decoded, axis=-1))
most_common_aa_accuracy = np.mean(np.argmax(seq_encoded.sum(axis=0), axis=-1) == np.argmax(seq_decoded.sum(axis=0), axis=-1))

In [24]:
print("Cross-entropy:", cross_entropy)
print("Same amino acid accuracy:", same_aa_accuracy)
print("Most common amino acid accuracy:", most_common_aa_accuracy)

Cross-entropy: 2.2488776995036415
Same amino acid accuracy: 0.7544910179640718
Most common amino acid accuracy: 0.7544910179640718


In [31]:
#decoding

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np

def one_hot_encoding(seq):
    # Define the amino acid alphabet
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY-'
    # Create a dictionary to map each amino acid to its index in the alphabet
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    # Initialize the one-hot encoded array
    one_hot = np.zeros((len(seq), len(amino_acids)))
    # Set the appropriate elements to 1
    for i, aa in enumerate(seq):
        if aa in aa_to_index:
            one_hot[i, aa_to_index[aa]] = 1
    return one_hot

def decode_sequence(one_hot_seq):
    # Define the amino acid alphabet
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY-'
    # Convert the one-hot encoded sequence back into an amino acid sequence
    return ''.join([amino_acids[i] for i in np.argmax(one_hot_seq, axis=-1)])

# Define the size of the latent space
latent_dim = 32

# Define the autoencoder architecture
input_seq = Input(shape=(None, len(amino_acids)))
encoded = Dense(latent_dim, activation='relu')(input_seq)
decoded = Dense(len(amino_acids), activation='softmax')(encoded)
autoencoder = Model(input_seq, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')


# Encode the sequence using one-hot encoding
seq_encoded = one_hot_encoding(seq).reshape(1, -1, len(amino_acids))

# Train the autoencoder on the encoded sequence
autoencoder.fit(seq_encoded, seq_encoded, epochs=100)




Epoch 1/100
1/1 [==============================] - 1s 619ms/step - loss: 3.0266
Epoch 2/100
1/1 [==============================] - 0s 10ms/step - loss: 3.0195
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 3.0124
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 3.0053
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 2.9983
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 2.9913
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 2.9843
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 2.9773
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: 2.9704
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 2.9634
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 2.9565
Epoch 12/100
1/1 [==============================] - 0s 13ms/step - loss: 2.9496
Epoch 13/100
1/1 [==============================

In [32]:
# Use the autoencoder to encode and decode the sequence
seq_decoded = autoencoder.predict(seq_encoded)

1/1 [==============================] - 0s 95ms/step


In [33]:
# Compute reconstruction error metrics
cross_entropy = -np.mean(np.sum(seq_encoded * np.log(seq_decoded), axis=-1))
same_aa_accuracy = np.mean(np.argmax(seq_encoded, axis=-1) == np.argmax(seq_decoded, axis=-1))
most_common_aa_accuracy = np.mean(np.argmax(seq_encoded.sum(axis=0), axis=-1) == np.argmax(seq_decoded.sum(axis=0), axis=-1))

print("Cross-Entropy:", cross_entropy)
print("Accuracy (Same aa):", same_aa_accuracy)
print("Accuracy (Mode aa):", most_common_aa_accuracy)


Cross-Entropy: 2.229750713902319
Accuracy (Same aa): 0.874251497005988
Accuracy (Mode aa): 0.874251497005988


In [34]:
# Decode and print the original and reconstructed sequences
print("Original sequence:")
print(decode_sequence(seq_encoded[0]))
print("Reconstructed sequence:")
print(decode_sequence(seq_decoded[0]))


Original sequence:
MDVMTYSDARAQLKGVMDRAIHDKQEVIVTRKKGESVVVVSLETWNAVNETLHLLSTPKNASRLRASIAQLDAGTGEERELTEMKLVFSDHAWEDYQYWVSTNDKVRARINELIKQCKRTPFKGTGKPEPLKGDLTGWWSRRISQEDRMVYRVDSQSLEIAQLRFHY
Reconstructed sequence:
MDVMTYSDARAQLKRVMDRAIHDKQEVIVTRKKRESVVVVSLETRNAVNETLHLLSTIKNASRLRASIAQLDARTREERELTEMKLVESDHAREDYQYRVSTNDKVRARINELIKQLKRTIEKRTRKIEILKRDLTRRRSRRISQEDRMVYRVDSQSLEIAQLREHY
